In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

from keras.layers import Dense, Flatten, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras import Sequential
import keras
from sklearn.preprocessing import OneHotEncoder

from keras.layers import *

E:\Anaconda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Loading cleaned data

In [2]:
train = pd.read_csv('salary_train_fixed.csv')
train.head()

,Category,ContractTime,ContractType,FullDescription,Id,SalaryNormalized,SalaryRaw,company,location_1,location_2,location_3,location_4,location_5,location_6,sourcename,title
0,8,1,0,Engineering Systems Analyst Dorking Surrey Sal...,12612628,25000,20000 - 30000/annum 20-30K,912,1,0,53,201,353,462,42,0
1,8,1,0,Stress Engineer Glasgow Salary **** to **** We...,12612830,30000,25000 - 35000/annum 25-35K,912,1,0,0,0,962,29,42,0
2,8,1,0,Mathematical Modeller / Simulation Analyst / O...,12612844,30000,20000 - 40000/annum 20-40K,912,1,0,0,0,1004,23,42,0
3,8,1,0,Engineering Systems Analyst / Mathematical Mod...,12613049,27500,25000 - 30000/annum 25K-30K negotiable,912,1,0,0,185,1066,11,42,0
4,8,1,0,"Pioneer, Miser Engineering Systems Analyst Do...",12613647,25000,20000 - 30000/annum 20-30K,912,1,0,0,185,1066,11,42,0


In [ ]:
test = pd.read_csv('salary_test_fixed.csv')
test.head()

** Full description to vector**

In [ ]:
full_des = pd.read_csv('./dataset/salay_first_dataset.csv',low_memory=False)['FullDescription']

In [ ]:
# integer encode the documents 独热编码
vocab_size = 1000
encoded_docs = [one_hot(d, vocab_size) for d in tqdm(full_des)]

In [ ]:
# pad documents to a max length of max words
maxlength = len(max(encoded_docs, key=len))
padded_docs = pad_sequences(encoded_docs, maxlen = maxlength, padding='post')
padded_docs.shape

In [ ]:
# normallized
padded_docs = padded_docs/vocab_size

In [ ]:
fullDes = padded_docs[122462:367230]
print(fullDes.shape)

** other features to one-hot vector**

In [3]:
Category = train['Category']
Category.shape

(244768,)

In [4]:
category_onehot = keras.utils.to_categorical(Category, len(set(Category))) 
category_onehot.shape

(244768, 29)

In [5]:
ContractTime = train['ContractTime']
ContractTime.shape

(244768,)

In [6]:
ContractTime_onehot = keras.utils.to_categorical(ContractTime, len(set(ContractTime))) 
ContractTime_onehot.shape

(244768, 3)

In [7]:
ContractType = train['ContractType']
ContractType.shape

(244768,)

In [8]:
ContractType_onehot = keras.utils.to_categorical(ContractType, len(set(ContractType))) 
ContractType_onehot.shape

(244768, 3)

In [9]:
location_1 = train['location_1']
location_1.shape

(244768,)

In [10]:
location_1_onehot = keras.utils.to_categorical(location_1, len(set(location_1))) 
location_1_onehot.shape

(244768, 2)

In [11]:
location_2 = train['location_2']
location_2.shape

(244768,)

In [12]:
location_2_onehot = keras.utils.to_categorical(location_2, len(set(location_2))) 
location_2_onehot.shape

(244768, 12)

In [13]:
location_3 = train['location_3']
location_3.shape

(244768,)

In [14]:
location_3_onehot = keras.utils.to_categorical(location_3, 74) 
location_3_onehot.shape

(244768, 74)

In [15]:
location_4 = train['location_4']
location_4.shape

(244768,)

In [16]:
location_4_onehot = keras.utils.to_categorical(location_4, 241) 
location_4_onehot.shape

(244768, 241)

In [17]:
location_5 = train['location_5']
location_5.shape

(244768,)

In [18]:
location_5_onehot = keras.utils.to_categorical(location_5, 1239) 
location_5_onehot.shape

(244768, 1239)

In [19]:
location_6 = train['location_6']
location_6.shape

(244768,)

In [20]:
location_6_onehot = keras.utils.to_categorical(location_6, len(set(location_6))) 
location_6_onehot.shape

(244768, 2088)

In [21]:
company = train['company']
company.shape

(244768,)

In [22]:
company_onehot = keras.utils.to_categorical(company, len(set(company))) 
company_onehot.shape

(244768, 3038)

In [23]:
sourcename = train['sourcename']
sourcename.shape

(244768,)

In [24]:
sourcename_onehot = keras.utils.to_categorical(sourcename, 169) 
sourcename_onehot.shape

(244768, 169)

In [25]:
title = train['title']
title.shape

(244768,)

In [26]:
title_onehot = keras.utils.to_categorical(title, len(set(title))) 
title_onehot.shape

(244768, 2838)

# choose trian features

In [27]:
# salay data
y_train = train['SalaryNormalized']
print(y_train.shape)

(244768,)


In [28]:
features = np.concatenate([title_onehot, location_1_onehot, location_2_onehot, location_3_onehot,
                          location_4_onehot, location_5_onehot, location_6_onehot, ContractType_onehot,
                          ContractTime_onehot, company_onehot, category_onehot, sourcename_onehot], axis=-1) # without FullDescription
features.shape

(244768, 9736)

In [29]:
#free memeroy
del train

In [30]:
# free memeory
del Category,category_onehot,ContractTime,ContractTime_onehot,ContractType,ContractType_onehot

In [31]:
# free memeory
del location_1,location_2,location_3,location_4,location_5,location_6,location_1_onehot,location_2_onehot,location_3_onehot,location_4_onehot,location_5_onehot,location_6_onehot

In [32]:
# free memeroy
del company,company_onehot, sourcename,sourcename_onehot, title,title_onehot

# build model

In [33]:
# create model
model = Sequential()
model.add(Dense(4000,input_dim=9736, activation='relu',use_bias=True))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='relu',use_bias=True))
model.add(Dropout(0.5))
model.add(Dense(1))
# Compile model
model.compile(loss='mean_squared_error', optimizer='rmsprop')
print(model.summary())

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4000)              38948000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              4001000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1001      
Total params: 42,950,001
Trainable params: 42,950,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
hist = model.fit(features, y_train, batch_size=1, epochs=100, shuffle=True,verbose=2,validation_split=0.5)  

Train on 122384 samples, validate on 122384 samples
Epoch 1/100
